In [7]:
!pip install category_encoders

     |████████████████████████████████| 81kB 5.0MB/s 


In [8]:
import pandas as pd
import numpy as np
import category_encoders as ce

# Disable warnings 
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
#casting data types
def casting(df):
  df['NUM_UNITS'] = df['NUM_UNITS'].astype(pd.Int64Dtype())
  df['KITCHENS'] = df['KITCHENS'].astype(pd.Int64Dtype())
  df['USECODE'] = df['USECODE'].astype(pd.Int64Dtype()).astype('category')
  df['ZIPCODE'] = df['ZIPCODE'].astype(pd.Int64Dtype()).astype('category')
  df['CMPLX_NUM'] = df['CMPLX_NUM'].astype(pd.Int64Dtype())
  df['CENSUS_TRACT'] = df['CMPLX_NUM'].astype(pd.Int64Dtype())
  df['SALEDATE'] = pd.to_datetime(df['SALEDATE'])
  return df

In [2]:
def saleyear(df):
  df['SALEYEAR'] = df['SALEDATE'].dt.year
  return df

In [3]:
def drop_qualified(df):
  df = df[df['QUALIFIED']=='Q']
  df = df.drop('QUALIFIED', axis=1)
  
  return df
  

In [4]:
def residential(df):
  df = df[df['SOURCE']=='Residential']
  
  return df

In [9]:
#AYB, EYB change to age, SQFT/ROOMS
def feature_eng(df):
  df['AYB_SALEYEAR_DIFF'] = df['SALEYEAR']-df['AYB']
  df['EYB_SALEYEAR_DIFF'] = np.where(df['SALEYEAR']<df['EYB'],0,df['SALEYEAR']-df['EYB'])
  df['SQFT_ROOMS'] = df['GBA']/df['ROOMS']

  return df

In [10]:
#feature engineering YR_RMDL
def yr_rmdl(df):
  df['RMDL'] = np.where(df['YR_RMDL'].isna(),0,1)
  df = df.drop('YR_RMDL',axis=1)
  return df

In [11]:
#drop unusual values
def drop_unusual(df):
  df['AC'] = np.where(df['AC']=='Y',1,0) #convert to 1 0 
  df = df[df['HEAT']!='No Data'] #delete no data
  df = df[df['ROOMS']!=0] #exclude property with 0 rooms
  df = df[df['AYB']<= df['SALEYEAR']] #exclude values where AYB > SALEYEAR
  df = df[df['AYB']<= df['EYB']] #exclude values where AYB > EYB
  df = df[~df['GRADE'].isin(['Exceptional-D','Exceptional-C','Exceptional-B',
                             'Exceptional-A'])] #exclude exceptional GRADE

  return df

In [12]:
#drop unused features : CMPLX_NUM,LIVING_GBA,CENSUS_TRACT,ASSESSMENT_SUBNBHD,
#                       FULLADDRESS,NATIONALGRID,CENSUS_BLOCK,CITY,STATE,X,Y,
#                       SALEDATE
#drop unrelated columns : GIS_LAST_MOD_DTTM, SOURCE
#drop similar values : STORIES
#drop unused location features : ZIPCODE,ASSESSMENT_NBHD,SQUARE,QUADRANT,
#                                LATITUDE,LONGITUDE
#drop weak correlation and multicollinearity : BLDG_NUM, NUM_UNITS
#drop based on model evaluation : ROOMS, USECODE
def drop_unused_features(df):
  df_clean = df.drop(['CMPLX_NUM','LIVING_GBA','CENSUS_TRACT',
                        'ASSESSMENT_SUBNBHD','FULLADDRESS','NATIONALGRID',
                        'CENSUS_BLOCK','CITY','STATE','X','Y','SALEDATE',
                        'GIS_LAST_MOD_DTTM','SOURCE','STORIES','ZIPCODE',
                        'ASSESSMENT_NBHD','SQUARE','QUADRANT','LATITUDE',
                        'LONGITUDE','BLDG_NUM','NUM_UNITS','ROOMS','USECODE'],
                        axis=1)
  return df_clean

In [13]:
#drop remaining missing values
def drop_all(df):
  df = df.dropna()
  return df

In [14]:
#encoding
def encoding_categorical(df):
  ordinal_mapping = [{'col':'CNDTN','mapping':{'Poor':1,'Fair':2,'Average':3,'Good':4,'Very Good':5,'Excellent':6}}]
  ordinal_encoder = ce.OrdinalEncoder(cols ='CNDTN',mapping = ordinal_mapping) 
  df = ordinal_encoder.fit_transform(df)                                                
  return df

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/elvanselvano/purwadhika-final-project/main/dataset/dev2.csv')
test = pd.read_csv('https://raw.githubusercontent.com/elvanselvano/purwadhika-final-project/main/dataset/test2.csv')

In [16]:
df1 = casting(df)
df2 = saleyear(df1)
df3 = drop_qualified(df2)
df4 = residential(df3)
df5 = feature_eng(df4)
df6 = yr_rmdl(df5)
df7 = drop_unusual(df6)
df8 = drop_unused_features(df7)
df9 = drop_all(df8)
df10 = encoding_categorical(df9)
df10.to_csv('dev_preproc.csv',index=False)

In [17]:
df = test.copy()
df1 = casting(df)
df2 = saleyear(df1)
df3 = drop_qualified(df2)
df4 = residential(df3)
df5 = feature_eng(df4)
df6 = yr_rmdl(df5)
df7 = drop_unusual(df6)
df8 = drop_unused_features(df7)
df9 = drop_all(df8)
df10 = encoding_categorical(df9)
df10.to_csv('test_preproc.csv',index=False)